In [1]:
!pip install torch transformers datasets scikit-learn pandas numpy tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
import numpy as np
import torch
from tqdm import tqdm
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, BertModel

from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from scipy.special import softmax

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd

In [6]:
# Load the IMDB dataset
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/senitment_for_NIC/IMDB Dataset.csv')

In [7]:
df['label'] = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

In [8]:
# Create a balanced sample of 200 (100 positive and 100 negative)
df_positive = df[df['label'] == 1].sample(100, random_state=42)
df_negative = df[df['label'] == 0].sample(100, random_state=42)
balanced_df = pd.concat([df_positive, df_negative])

In [9]:
# Extract the texts and labels
texts = balanced_df['review'].tolist()
labels = balanced_df['label'].tolist()

## Use BERT for Sentiment Prediction

In [10]:
# Load the pre-trained BERT tokenizer and model for sequence classification


MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
bert_model = AutoModelForSequenceClassification.from_pretrained(MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
def return_sentiment(txt):
    encoded_input = tokenizer(txt, return_tensors='pt', padding=True, truncation=True, max_length=510)
    output = bert_model(**encoded_input)
    score = output[0][0].detach().numpy()
    scores = softmax(score)

    # Get the predicted class (0 for negative, 1 for positive)
    if scores[2] > scores[0]:
      return 1

    return 0

In [ ]:
# Predict using BERT
predictions = []

for text in tqdm(texts):
  predictions.append(return_sentiment(text))


100%|██████████| 200/200 [03:57<00:00,  1.19s/it]


In [ ]:
# Create a confusion matrix for BERT model predictions
print("Confusion Matrix for BERT model:")
bert_conf_matrix = confusion_matrix(labels, predictions)
print(bert_conf_matrix)
print(classification_report(labels, predictions))

Confusion Matrix for BERT model:
[[87 13]
 [24 76]]
              precision    recall  f1-score   support

           0       0.78      0.87      0.82       100
           1       0.85      0.76      0.80       100

    accuracy                           0.81       200
   macro avg       0.82      0.81      0.81       200
weighted avg       0.82      0.81      0.81       200



## Extract BERT Embeddings and Use Logistic Regression

In [ ]:
# Extract word embeddings from BERT and use Logistic Regression
lrtokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the texts
inputs = lrtokenizer(texts, padding=True, truncation=True, return_tensors="pt")
bert_model = BertModel.from_pretrained("bert-base-uncased")

inputs = lrtokenizer(texts, padding=True, truncation=True, return_tensors="pt",max_length=510)
with torch.no_grad():
    embeddings = bert_model(**inputs).last_hidden_state[:, 0, :].numpy()  # Take the [CLS] token

In [ ]:
# Split the embeddings for training and testing the logistic regression
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)


In [ ]:
# Train a logistic regression model
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

In [ ]:
# Predict using the logistic regression model
lr_predictions = lr_model.predict(X_test)

In [ ]:
# Create a confusion matrix for logistic regression predictions
print("Confusion Matrix for Logistic Regression model:")
lr_conf_matrix = confusion_matrix(y_test, lr_predictions)
print(lr_conf_matrix)
print(classification_report(y_test, lr_predictions))